In [1]:
import openai
import os
from dotenv import load_dotenv

from typing import Union, Iterator

# This line loads the variables from your .env file
load_dotenv()

# os.getenv() securely reads the environment variable
api_key = os.getenv("OPENAI_API_KEY")


def adaptive_ai_response(prompt: str) -> Union[str, Iterator[str]]:
    """Automatically chooses to stream or not based on the prompt."""

    # Heuristic: If the prompt asks for long-form content, enable streaming.
    streaming_keywords = ["write", "explain", "describe", "story", "report", "detailed"]
    should_stream = any(keyword in prompt.lower() for keyword in streaming_keywords)

    client = openai.OpenAI(api_key=api_key, base_url="https://openrouter.ai/api/v1")

    if should_stream:
        print("-> Using streaming response.")
        stream = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "user", "content": prompt}],
            stream=True,
        )
        # Return the generator object for the client to iterate over
        return (chunk.choices[0].delta.content or "" for chunk in stream)
    else:
        print("-> Using batch response.")
        response = client.chat.completions.create(
            model="gpt-4o-mini", messages=[{"role": "user", "content": prompt}]
        )
        return response.choices[0].message.content


# Test with a short query
result = adaptive_ai_response("What is the capital of Japan?")
print(result)

# Test with a long query
stream_result = adaptive_ai_response("Write a short story about a brave knight.")
for chunk in stream_result:
    print(chunk, end="", flush=True)
print()


-> Using batch response.
The capital of Japan is Tokyo.
-> Using streaming response.
In the Kingdom of Eldoria, where emerald hills kissed the sky and rivers shimmered like silver threads, there lived a brave knight named Sir Cedric. Known throughout the realm for his valiant heart and unwavering sense of justice, Cedric wore his shining armor like a second skin, and his sword, named "Valor's Edge," was said to contain the strength of a thousand men.

One fateful day, as the sun dipped below the horizon, casting a golden glow over the land, the kingdom was shaken by an ominous threat. A fearsome dragon, scales as black as night and eyes like molten gold, had emerged from the eastern mountains. It descended upon the villages, scorching fields and carrying off livestock, instilling fear into the hearts of even the bravest souls.

The King summoned his knights, each one trembling at the thought of facing the beast. Yet, Sir Cedric stood tall among them, his blue eyes blazing with determin